In [62]:
import pandas as pd
import numpy as np
import re

In [63]:
hn = pd.read_csv("C:/Users/obalana prosper/Videos/New folder (3)/hacker_news.csv")
print("hn added successfully")

hn added successfully


In [64]:
titles = hn['title']

In [65]:
pattern = r'[Pp]ython'
python_counts = titles.str.contains(pattern).sum()
print(python_counts)

160


re.I the ignore case flag to make it case insensitive

In [66]:
pattern = r'python'
python_counts = titles.str.contains(pattern, flags = re.I).sum()
print(python_counts)

160


In [67]:
pattern = r'[Ss][Qq][Ll]'
sql_counts = titles.str.contains(pattern).sum()
print(sql_counts)

108


In [68]:
pattern = r'sql'
sql_counts = titles.str.contains(pattern, flags = re.I).sum()
print(sql_counts)

108


### Capture Groups

#### Create a frequency

In [69]:
pattern = r'(SQL)'
sql_capitalizations = titles.str.extract(pattern,flags = re.I,expand = False)
sql_capitalizations_freq = sql_capitalizations.value_counts()
print(sql_capitalizations_freq)

SQL    101
Sql      4
sql      3
Name: title, dtype: int64


In [70]:
pattern = r'(\w+SQL)'
sql_flavors = titles.str.extract(pattern, flags = re.I, expand = False)
sql_flavors_freq = sql_flavors.value_counts() 
print(sql_flavors_freq)

PostgreSQL    27
NoSQL         16
MySQL         12
SparkSQL       1
nosql          1
CloudSQL       1
MemSQL         1
mySql          1
Name: title, dtype: int64


In [89]:
hn_sql = hn[hn['title'].str.contains(r'\w+SQL', flags = re.I)].copy()

In [93]:
hn_sql['flavor'] = titles.str.extract(pattern, flags = re.I)

In [94]:
hn_sql['flavor'] = hn_sql['flavor'].str.lower()

In [95]:
sql_pivot = hn_sql.pivot_table(index = 'flavor',values = 'num_comments')

In [96]:
sql_pivot

,num_comments
flavor,
cloudsql,5.000000
memsql,14.000000
mysql,12.230769
nosql,14.529412
postgresql,25.962963
sparksql,1.000000


### Using a capture group to extract data

[\d\.]+ - one or more digit or . characters

In [129]:
pattern = r'python ([\d\.]+)'

In [130]:
python_extract = titles.str.extract(pattern, flags = re.I, expand = False)
python_counts = python_extract.value_counts()
print(python_counts)

3        10
2         3
3.5       3
3.6       2
3.5.0     1
4         1
8         1
2.7       1
1.5       1
Name: title, dtype: int64


In [131]:
python_dict = dict(python_counts)

In [132]:
print(python_dict)

{'3': 10, '2': 3, '3.5': 3, '3.6': 2, '3.5.0': 1, '4': 1, '8': 1, '2.7': 1, '1.5': 1}


### Counting Mentions of the 'C' language

In [133]:
def first_10_matches(pattern):
    all_matches = titles[titles.str.contains(pattern)]
    first_10 = all_matches.head(10)
    return first_10

In [134]:
first_10_matches(r'\b[Cc]\b')

14                 Custom Deleters for C++ Smart Pointers
221                        Lisp, C++: Sadness in my heart
222                  MemSQL (YC W11) Raises $36M Series C
354     VW C.E.O. Personally Apologized to President O...
366                      The new C standards are worth it
445           Moz raises $10m Series C from Foundry Group
509     BDE 3.0 (Bloomberg's core C++ library): Open S...
522          Fuchsia: Micro kernel written in C by Google
550     How to Become a C.E.O.? The Quickest Path Is a...
1283    A lightweight C++ signals and slots implementa...
Name: title, dtype: object

In [148]:
first_10_matches(r'\b[Cc]\b[^.+]')

366                      The new C standards are worth it
445           Moz raises $10m Series C from Foundry Group
522          Fuchsia: Micro kernel written in C by Google
1308            Show HN: Yupp, yet another C preprocessor
1327                     The C standard formalized in Coq
1366                          GNU C Library 2.23 released
1430    Cysignals: signal handling (SIGINT, SIGSEGV, )...
1621                        SDCC  Small Device C Compiler
1950    Rewriting a Ruby C Extension in Rust: How a Na...
2196    MyHTML  HTML Parser on Pure C with POSIX Threa...
Name: title, dtype: object

### Using Lookarounds to Control Matches Based on Surrounding Text

1. Positive lookaround = zzz(?=abc) : Matches zzz only when it is followed by abc
2. Negative Lookaround = zzz(?!abc) : Matches zzz only when it is not followed by abc
3. Positive Lookbehind = (?<=abc)zzz : Matches zzz only when it's preceeded by abc
4. Negative Lookbehind = (?<!abc)zzz : Matches zzz only when it's not preceeded by abc

In [149]:
test_cases = ['Red_Green_Blue','Yellow_Green_Red','Red_Green_Red','Yellow_Green_Blue','Green']

In [150]:
def run_test_cases(pattern):
    for tc in test_cases:
        result = re.search(pattern, tc)
        print(result or 'NO MATCH')

In [152]:
run_test_cases(r'Green(?=_Blue)')

<_sre.SRE_Match object; span=(4, 9), match='Green'>
NO MATCH
NO MATCH
<_sre.SRE_Match object; span=(7, 12), match='Green'>
NO MATCH


In [155]:
run_test_cases(r'(?<=Red_)Green')

<_sre.SRE_Match object; span=(4, 9), match='Green'>
NO MATCH
<_sre.SRE_Match object; span=(4, 9), match='Green'>
NO MATCH
NO MATCH


In [157]:
run_test_cases(r'(?<!Yellow_)Green')

<_sre.SRE_Match object; span=(4, 9), match='Green'>
NO MATCH
<_sre.SRE_Match object; span=(4, 9), match='Green'>
NO MATCH
<_sre.SRE_Match object; span=(0, 5), match='Green'>


In [161]:
run_test_cases(r'Green(?=.{5})')

<_sre.SRE_Match object; span=(4, 9), match='Green'>
NO MATCH
NO MATCH
<_sre.SRE_Match object; span=(7, 12), match='Green'>
NO MATCH


In [169]:
first_10_matches(r'(?<!Series)\b[Cc](?=\W)\b[^.+]')

366                      The new C standards are worth it
445           Moz raises $10m Series C from Foundry Group
522          Fuchsia: Micro kernel written in C by Google
1308            Show HN: Yupp, yet another C preprocessor
1327                     The C standard formalized in Coq
1366                          GNU C Library 2.23 released
1430    Cysignals: signal handling (SIGINT, SIGSEGV, )...
1621                        SDCC  Small Device C Compiler
1950    Rewriting a Ruby C Extension in Rust: How a Na...
2196    MyHTML  HTML Parser on Pure C with POSIX Threa...
Name: title, dtype: object

In [178]:
pattern = (r'(?<!Series)\b([Cc])(?=\W)\b[^.+]')
c_pattern = titles.str.extract(pattern, expand = False)
c_mentions = c_pattern.value_counts()
print(c_mentions)

C    81
c     3
Name: title, dtype: int64


In [229]:
pattern = (r'(?<!Series\s)\b([Cc])\b(?!w+)(?![\.\+])')
c_pattern = titles.str.contains(pattern)
c_mentions = c_pattern.sum()
print(c_mentions)

102


C:\Users\obalana prosper\Videos\New folder\New folder\lib\site-packages\ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [226]:
pattern = (r'(?<!Series\s)\b([Cc])\b(?![\.\+])')
c_pattern = titles.str.contains(pattern)
c_mentions = c_pattern.sum()
print(c_mentions)

102


C:\Users\obalana prosper\Videos\New folder\New folder\lib\site-packages\ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


### BackReference: Using Capture Groups in a RegEx Pattern

In [233]:
test_cases = ["I'm going to read a book.",
             "Green is my favourite color",
             "I love football",
             "No doubles here",
             "My name is Aaron"]

for tc in test_cases:
    print(re.search(r'(\w)\1',tc))


<_sre.SRE_Match object; span=(21, 23), match='oo'>
<_sre.SRE_Match object; span=(2, 4), match='ee'>
<_sre.SRE_Match object; span=(8, 10), match='oo'>
None
None


In [236]:
test_cases = pd.Series(test_cases)
print(test_cases.str.contains(r'(\w)\1'))

0     True
1     True
2     True
3    False
4    False
dtype: bool


C:\Users\obalana prosper\Videos\New folder\New folder\lib\site-packages\ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [279]:
pattern = (r'\b(\w+)\s\1\b')
repeated_words = titles.str.contains(pattern)

C:\Users\obalana prosper\Videos\New folder\New folder\lib\site-packages\ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [280]:
titles[repeated_words]

3103                  Silicon Valley Has a Problem Problem
3177                Wire Wire: A West African Cyber Threat
3179                         Flexbox Cheatsheet Cheatsheet
4798                            The Mindset Mindset (2015)
7277     Valentine's Day Special: Bye Bye Tinder, Flirt...
10372    Mcdonalds copying cyriak  cows cows cows in th...
11576                                    Bang Bang Control
11902          Cordless Telephones: Bye Bye Privacy (1991)
12698          Solving the the Monty-Hall-Problem in Swift
15050    Bye Bye Webrtc2SIP: WebRTC with Asterisk and A...
15840          Intellij-Rust Rust Plugin for IntelliJ IDEA
Name: title, dtype: object

### Substituting Regular Expression Matches

 re.sub(pattern, repl, string, flags = 0)

In [286]:
string = 'aBcDEfGHIj'
print(re.sub(r'[A-Z]','-',string))

a-c--f---j


In [287]:
sql_variations = pd.Series(['SQL','Sql','sql'])
sql_uniform = sql_variations.str.replace(r'sql','SQL',flags = re.I)
print(sql_uniform)

0    SQL
1    SQL
2    SQL
dtype: object


In [288]:
email_variations = pd.Series(['email','Email','eMail','EMAIL','e mail','E-mail','e-mail','E-Mail','emails','Emails','E-Mails'])

In [301]:
email_uniform = email_variations.str.replace(r'email','email',flags = re.I)
email_too = email_uniform.str.replace(r'e mail','email',flags = re.I)
email_var = email_too.str.replace(r'e-mail','email',flags = re.I)

In [302]:
email_var

0      email
1      email
2      email
3      email
4      email
5      email
6      email
7      email
8     emails
9     emails
10    emails
dtype: object

In [303]:
email_uniform = titles.str.replace(r'email','email',flags = re.I)
email_too = email_uniform.str.replace(r'e mail','email',flags = re.I)
titles_clean = email_too.str.replace(r'e-mail','email',flags = re.I)

In [304]:
titles_clean

0                                Interactive Dynamic Video
1        How to Use Open Source and Shut the Fuck Up at...
2        Florida DJs May Face Felony for April Fools' W...
3             Technology ventures: From Idea to Enterprise
4        Note by Note: The Making of Steinway L1037 (2007)
5        Title II kills investment? Comcast and other I...
6                           Nuts and Bolts Business Advice
7              Ask HN: How to improve my personal website?
8        Shims, Jigs and Other Woodworking Concepts to ...
9                                   That self-appendectomy
10       Crate raises $4M seed round for its next-gen S...
11       Advertising Cannot Maintain the Internet. Here...
12                                          Coding Is Over
13       Show HN: Wio Link  ESP8266 Based Web of Things...
14                  Custom Deleters for C++ Smart Pointers
15              How often to update third party libraries?
16                        Review my AI based marketing b

In [324]:
pattern = r'e( )?(-)?mail'

In [332]:
email_uniform = email_variations.str.replace(pattern,'email',flags = re.I)
email_uniform

0      email
1      email
2      email
3      email
4      email
5      email
6      email
7      email
8     emails
9     emails
10    emails
dtype: object

In [330]:
pattern = r'e[\-\s]?mail'

In [331]:
email_uniform = email_variations.str.replace(pattern,'email',flags = re.I)
email_uniform

0      email
1      email
2      email
3      email
4      email
5      email
6      email
7      email
8     emails
9     emails
10    emails
dtype: object

In [328]:
titles_clean = titles.str.replace(pattern,'email',flags = re.I)

In [329]:
titles_clean

0                                Interactive Dynamic Video
1        How to Use Open Source and Shut the Fuck Up at...
2        Florida DJs May Face Felony for April Fools' W...
3             Technology ventures: From Idea to Enterprise
4        Note by Note: The Making of Steinway L1037 (2007)
5        Title II kills investment? Comcast and other I...
6                           Nuts and Bolts Business Advice
7              Ask HN: How to improve my personal website?
8        Shims, Jigs and Other Woodworking Concepts to ...
9                                   That self-appendectomy
10       Crate raises $4M seed round for its next-gen S...
11       Advertising Cannot Maintain the Internet. Here...
12                                          Coding Is Over
13       Show HN: Wio Link  ESP8266 Based Web of Things...
14                  Custom Deleters for C++ Smart Pointers
15              How often to update third party libraries?
16                        Review my AI based marketing b

### Extracting Domains from URL

In [376]:
test_urls = pd.Series([
 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
 'http://www.interactivedynamicvideo.com/',
 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
 'http://evonomics.com/advertising-cannot-maintain-internet-heres-solution/',
 'HTTPS://github.com/keppel/pinn',
 'Http://phys.org/news/2015-09-scale-solar-youve.html',
 'https://iot.seeed.cc',
 'http://www.bfilipek.com/2016/04/custom-deleters-for-c-smart-pointers.html',
 'http://beta.crowdfireapp.com/?beta=agnipath',
 'https://www.valid.ly?param',
 'http://css-cursor.techstream.org'
])


In [382]:
v = 'http://css-cursor.techstream.org'
pattern = r'https?://([\w\-\.]+)'
test_urls.str.extract(pattern, flags = re.I)

,0
0,www.amazon.com
1,www.interactivedynamicvideo.com
2,www.nytimes.com
3,evonomics.com
4,github.com
5,phys.org
6,iot.seeed.cc
7,www.bfilipek.com
8,beta.crowdfireapp.com
9,www.valid.ly


In [384]:
domains = hn['url'].str.extract(pattern, flags = re.I, expand = False)

In [385]:
top_domains = domains.value_counts()

In [386]:
top_domains

github.com                       1008
medium.com                        825
www.nytimes.com                   525
www.theguardian.com               248
techcrunch.com                    245
www.youtube.com                   213
www.bloomberg.com                 193
arstechnica.com                   191
www.washingtonpost.com            190
www.wsj.com                       138
www.theatlantic.com               137
www.bbc.com                       134
www.wired.com                     114
www.theverge.com                  112
www.bbc.co.uk                     108
en.wikipedia.org                  100
twitter.com                        93
qz.com                             85
motherboard.vice.com               82
www.newyorker.com                  81
www.forbes.com                     78
nautil.us                          77
www.businessinsider.com            73
www.nature.com                     72
www.reuters.com                    71
www.economist.com                  66
arxiv.org   

### Extracting URL Parts Using Multiple Capture Groups

In [387]:
created_at = hn['created_at'].head()
print(created_at)

0     8/4/2016 11:52
1    1/26/2016 19:30
2    6/23/2016 22:20
3     6/17/2016 0:01
4     9/30/2015 4:12
Name: created_at, dtype: object


(.+)\s(.+) start group(
1. .+ one or more characters

In [389]:
pattern = r'(.+)\s(.+)'
created_at.str.extract(pattern)

,0,1
0,8/4/2016,11:52
1,1/26/2016,19:30
2,6/23/2016,22:20
3,6/17/2016,0:01
4,9/30/2015,4:12


In [394]:
test_urls = pd.Series([
 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
 'http://www.interactivedynamicvideo.com/',
 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
 'http://evonomics.com/advertising-cannot-maintain-internet-heres-solution/',
 'HTTPS://github.com/keppel/pinn',
 'Http://phys.org/news/2015-09-scale-solar-youve.html',
 'https://iot.seeed.cc',
 'http://www.bfilipek.com/2016/04/custom-deleters-for-c-smart-pointers.html',
 'http://beta.crowdfireapp.com/?beta=agnipath',
 'https://www.valid.ly?param',
 'http://css-cursor.techstream.org'
])

In [601]:
pattern = r'(https?)://([\w\-\.]+)/?(.*)'

In [603]:
test_urls.str.extract(pattern, flags = re.I)

,0,1,2
0,https,www.amazon.com,Technology-Ventures-Enterprise-Thomas-Byers/dp...
1,http,www.interactivedynamicvideo.com,
2,http,www.nytimes.com,2007/11/07/movies/07stein.html?_r=0
3,http,evonomics.com,advertising-cannot-maintain-internet-heres-sol...
4,HTTPS,github.com,keppel/pinn
5,Http,phys.org,news/2015-09-scale-solar-youve.html
6,https,iot.seeed.cc,
7,http,www.bfilipek.com,2016/04/custom-deleters-for-c-smart-pointers.html
8,http,beta.crowdfireapp.com,?beta=agnipath
9,https,www.valid.ly,?param


In [604]:
do = hn['url'].str.extract(pattern, expand = False, flags = re.I)

In [605]:
do

,0,1,2
0,http,www.interactivedynamicvideo.com,
1,http,hueniverse.com,2016/01/26/how-to-use-open-source-and-shut-the...
2,http,www.thewire.com,entertainment/2013/04/florida-djs-april-fools-...
3,https,www.amazon.com,Technology-Ventures-Enterprise-Thomas-Byers/dp...
4,http,www.nytimes.com,2007/11/07/movies/07stein.html?_r=0
5,http,arstechnica.com,business/2015/10/comcast-and-other-isps-boost-...
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,http,firstround.com,review/shims-jigs-and-other-woodworking-concep...
9,http,www.southpolestation.com,trivia/igy1/appendix.html


### Using Named Capture Groups To Extract Data

(?P<name>.+)\s(?P<time>.+)

In [591]:
created_at = hn['created_at'].head()
print(created_at)

0     8/4/2016 11:52
1    1/26/2016 19:30
2    6/23/2016 22:20
3     6/17/2016 0:01
4     9/30/2015 4:12
Name: created_at, dtype: object


In [594]:
pattern = r'(?P<date>.+)\s(?P<time>.+)'
created_at.str.extract(pattern)

,date,time
0,8/4/2016,11:52
1,1/26/2016,19:30
2,6/23/2016,22:20
3,6/17/2016,0:01
4,9/30/2015,4:12


In [608]:
pattern = r'(?P<protocol>https?)://(?P<domain>[\w\-\.]+)/?(?P<path>.*)'
test_urls.str.extract(pattern, flags = re.I)

,protocol,domain,path
0,https,www.amazon.com,Technology-Ventures-Enterprise-Thomas-Byers/dp...
1,http,www.interactivedynamicvideo.com,
2,http,www.nytimes.com,2007/11/07/movies/07stein.html?_r=0
3,http,evonomics.com,advertising-cannot-maintain-internet-heres-sol...
4,HTTPS,github.com,keppel/pinn
5,Http,phys.org,news/2015-09-scale-solar-youve.html
6,https,iot.seeed.cc,
7,http,www.bfilipek.com,2016/04/custom-deleters-for-c-smart-pointers.html
8,http,beta.crowdfireapp.com,?beta=agnipath
9,https,www.valid.ly,?param


In [598]:
do = hn['url'].str.extract(pattern, expand = False, flags = re.I)
do

,protocol,domain,path
0,http,www.interactivedynamicvideo.com,
1,http,hueniverse.com,2016/01/26/how-to-use-open-source-and-shut-the...
2,http,www.thewire.com,entertainment/2013/04/florida-djs-april-fools-...
3,https,www.amazon.com,Technology-Ventures-Enterprise-Thomas-Byers/dp...
4,http,www.nytimes.com,2007/11/07/movies/07stein.html?_r=0
5,http,arstechnica.com,business/2015/10/comcast-and-other-isps-boost-...
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,http,firstround.com,review/shims-jigs-and-other-woodworking-concep...
9,http,www.southpolestation.com,trivia/igy1/appendix.html
